# PPML para Modelos Gravitacionais de Comércio

## Introdução

Este tutorial demonstra o uso de **Poisson Pseudo-Maximum Likelihood (PPML)** para estimar modelos gravitacionais de comércio internacional.

### Por que PPML?

Santos Silva & Tenreyro (2006) demonstraram que o tradicional método OLS com log da variável dependente sofre de sérios problemas:

1. **Jensen's Inequality**: $E[\log(y)] \neq \log(E[y])$ → viés na presença de heteroskedasticidade
2. **Zeros**: Log não pode lidar com zeros (problema comum em dados de comércio)
3. **Heteroskedasticidade**: OLS(log) é inconsistente sob heteroskedasticidade

**PPML resolve todos esses problemas**:
- Consistente sob heteroskedasticidade (propriedade QML)
- Lida naturalmente com zeros
- Especificação correta de $E[y|X]$ ao invés de $E[\log(y)|X]$

### Referências

- Santos Silva, J.M.C., & Tenreyro, S. (2006). "The Log of Gravity." *Review of Economics and Statistics*, 88(4), 641-658.
- Head, K., & Mayer, T. (2014). "Gravity Equations: Workhorse, Toolkit, and Cookbook." *Handbook of International Economics*.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from panelbox.models.count import PPML
from statsmodels.regression.linear_model import OLS

np.random.seed(2024)

## 1. Simulação de Dados de Comércio Bilateral

Vamos simular dados de comércio bilateral seguindo um modelo gravitacional padrão:

$$
Trade_{ijt} = \exp(\beta_0 + \beta_1 \log(GDP_i) + \beta_2 \log(GDP_j) + \beta_3 \log(Distance_{ij}) + \epsilon_{ijt})
$$

Onde:
- $Trade_{ijt}$: Fluxo de comércio do país $i$ para país $j$ no ano $t$
- $GDP_i$, $GDP_j$: PIB dos países origem e destino
- $Distance_{ij}$: Distância entre países
- $\epsilon_{ijt}$: Erro aleatório (com heteroskedasticidade)

In [ ]:
# Parâmetros da simulação
n_countries = 15
n_years = 8

# Criar pares de países (excluindo auto-comércio)
data = []
for year in range(n_years):
    for i in range(n_countries):
        for j in range(n_countries):
            if i != j:  # Sem auto-comércio
                data.append({
                    'year': 2010 + year,
                    'origin': i,
                    'dest': j,
                    'pair_id': i * n_countries + j
                })

df = pd.DataFrame(data)
print(f"Total de observações: {len(df)}")
print(f"Pares únicos: {df['pair_id'].nunique()}")

In [ ]:
# Gerar PIBs com crescimento ao longo do tempo
gdp_base_2010 = np.random.uniform(20, 28, n_countries)  # Log do PIB base
gdp_growth_rate = np.random.uniform(0.01, 0.05, n_countries)  # Taxa de crescimento anual

df['log_gdp_origin'] = df.apply(
    lambda x: gdp_base_2010[int(x['origin'])] + gdp_growth_rate[int(x['origin'])] * (x['year'] - 2010),
    axis=1
)

df['log_gdp_dest'] = df.apply(
    lambda x: gdp_base_2010[int(x['dest'])] + gdp_growth_rate[int(x['dest'])] * (x['year'] - 2010),
    axis=1
)

# Gerar distâncias (invariantes no tempo)
distance_matrix = np.random.uniform(4, 9, (n_countries, n_countries))
np.fill_diagonal(distance_matrix, 0)

df['log_distance'] = df.apply(
    lambda x: distance_matrix[int(x['origin']), int(x['dest'])],
    axis=1
)

# Estatísticas descritivas
print("\nEstatísticas das variáveis:")
print(df[['log_gdp_origin', 'log_gdp_dest', 'log_distance']].describe())

In [ ]:
# Parâmetros verdadeiros do modelo gravitacional
beta_0 = -12  # Intercept
beta_gdp_origin = 0.9  # Elasticidade PIB origem
beta_gdp_dest = 0.8    # Elasticidade PIB destino
beta_distance = -1.1   # Elasticidade distância (negativa)

# Preditor linear
linear_pred = (
    beta_0 +
    beta_gdp_origin * df['log_gdp_origin'] +
    beta_gdp_dest * df['log_gdp_dest'] +
    beta_distance * df['log_distance']
)

# Adicionar HETEROSKEDASTICIDADE (crucial para demonstrar vantagem do PPML)
# Variância aumenta com a distância
hetero_factor = 0.3 + 0.05 * df['log_distance']
epsilon = np.random.randn(len(df)) * hetero_factor

# Gerar fluxos de comércio via Poisson
lambda_it = np.exp(linear_pred + epsilon)
df['trade'] = np.random.poisson(lambda_it)

# Introduzir ZEROS (realista para comércio)
# 12% de zeros (comum em dados de comércio bilateral)
zero_mask = np.random.rand(len(df)) < 0.12
df.loc[zero_mask, 'trade'] = 0

n_zeros = (df['trade'] == 0).sum()
pct_zeros = 100 * n_zeros / len(df)

print(f"\nFluxos de comércio:")
print(f"  Zeros: {n_zeros} ({pct_zeros:.1f}%)")
print(f"  Média: {df['trade'].mean():.2f}")
print(f"  Mediana: {df['trade'].median():.2f}")
print(f"  Máximo: {df['trade'].max()}")

In [ ]:
# Visualizar distribuição dos fluxos de comércio
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribuição completa
axes[0].hist(df['trade'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Fluxo de Comércio')
axes[0].set_ylabel('Frequência')
axes[0].set_title(f'Distribuição dos Fluxos de Comércio\n({pct_zeros:.1f}% zeros)')
axes[0].axvline(0, color='red', linestyle='--', label='Zeros')
axes[0].legend()

# Distribuição sem zeros (em log)
trade_positive = df[df['trade'] > 0]['trade']
axes[1].hist(np.log(trade_positive), bins=40, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Log(Fluxo de Comércio)')
axes[1].set_ylabel('Frequência')
axes[1].set_title('Distribuição Log (apenas valores positivos)')

plt.tight_layout()
plt.show()

## 2. Estimação via PPML

PPML estima o modelo gravitacional de forma consistente mesmo na presença de:
- Heteroskedasticidade
- Zeros na variável dependente
- Erros multiplicativos

In [ ]:
# Preparar dados para PPML
y_ppml = df['trade'].values
X_ppml = df[['log_gdp_origin', 'log_gdp_dest', 'log_distance']].values
X_ppml = np.column_stack([np.ones(len(X_ppml)), X_ppml])

# Estimar PPML (pooled)
model_ppml = PPML(
    endog=y_ppml,
    exog=X_ppml,
    entity_id=df['pair_id'].values,
    fixed_effects=False,
    exog_names=['const', 'log_gdp_origin', 'log_gdp_dest', 'log_distance']
)

result_ppml = model_ppml.fit()

print("\n" + "="*60)
print("PPML ESTIMATION RESULTS")
print("="*60)
print(result_ppml.summary())

## 3. Comparação com OLS(log)

Vamos comparar PPML com o método tradicional OLS em log, que:
1. Precisa **descartar zeros**
2. É **inconsistente** sob heteroskedasticidade
3. Sofre de **viés de Jensen**

In [ ]:
# OLS: precisa remover zeros
df_no_zeros = df[df['trade'] > 0].copy()
print(f"OLS perde {len(df) - len(df_no_zeros)} observações ({pct_zeros:.1f}%) por causa de zeros!\n")

# Preparar dados para OLS
y_ols = np.log(df_no_zeros['trade'].values)
X_ols = df_no_zeros[['log_gdp_origin', 'log_gdp_dest', 'log_distance']].values
X_ols = np.column_stack([np.ones(len(X_ols)), X_ols])

# Estimar OLS
model_ols = OLS(y_ols, X_ols)
result_ols = model_ols.fit(cov_type='cluster', cov_kwds={'groups': df_no_zeros['pair_id'].values})

print("="*60)
print("OLS(log) ESTIMATION RESULTS")
print("="*60)
print(result_ols.summary())

In [ ]:
# Comparação direta dos coeficientes
comparison = result_ppml.compare_with_ols(result_ols)

print("\n" + "="*70)
print("PPML vs OLS(log) - COMPARISON")
print("="*70)
print(comparison)
print("\n" + "="*70)

In [ ]:
# Visualizar comparação
fig, ax = plt.subplots(figsize=(10, 6))

var_names = ['Intercept', 'log(GDP origin)', 'log(GDP dest)', 'log(Distance)']
true_params = [beta_0, beta_gdp_origin, beta_gdp_dest, beta_distance]

x = np.arange(len(var_names))
width = 0.25

ax.bar(x - width, true_params, width, label='True Parameters', color='gray', alpha=0.7)
ax.bar(x, result_ppml.params, width, label='PPML', color='blue', alpha=0.7)
ax.bar(x + width, result_ols.params, width, label='OLS(log)', color='red', alpha=0.7)

ax.set_xlabel('Variables')
ax.set_ylabel('Coefficient Value')
ax.set_title('PPML vs OLS(log): Parameter Recovery')
ax.set_xticks(x)
ax.set_xticklabels(var_names, rotation=15, ha='right')
ax.legend()
ax.axhline(0, color='black', linewidth=0.5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Interpretação de Elasticidades

No modelo gravitacional com variáveis em log, os coeficientes $\beta$ são **elasticidades**:

$$
\epsilon = \frac{\partial \log(E[Trade])}{\partial \log(X)} = \frac{\partial E[Trade] / E[Trade]}{\partial X / X}
$$

Interpretação:
- $\beta_{GDP} = 0.9$ → 1% aumento no PIB → 0.9% aumento no comércio
- $\beta_{Distance} = -1.1$ → 1% aumento na distância → 1.1% queda no comércio

In [ ]:
# Tabela de elasticidades (PPML)
elast_table = result_ppml.elasticities()

print("\n" + "="*60)
print("PPML ELASTICITIES")
print("="*60)
print(elast_table)
print("\nInterpretação:")
print(f"  - 1% ↑ PIB origem  → {elast_table.iloc[1]['elasticity']:.3f}% ↑ comércio")
print(f"  - 1% ↑ PIB destino → {elast_table.iloc[2]['elasticity']:.3f}% ↑ comércio")
print(f"  - 1% ↑ distância   → {elast_table.iloc[3]['elasticity']:.3f}% ↓ comércio")

In [ ]:
# Elasticidade específica para uma variável
elast_gdp_origin = result_ppml.elasticity('log_gdp_origin')

print("\n" + "="*60)
print("Elasticidade detalhada: PIB Origem")
print("="*60)
for key, value in elast_gdp_origin.items():
    print(f"  {key}: {value}")

## 5. PPML com Fixed Effects

Em modelos gravitacionais, é comum incluir fixed effects de pares (pair FE) para controlar fatores bilaterais não observados.

Com pair FE, a distância (time-invariant) é absorvida, então estimamos apenas efeitos do PIB.

In [ ]:
# PPML com Fixed Effects de pares
# Note: distância é time-invariant, então será absorvida pelo FE

y_ppml_fe = df['trade'].values
X_ppml_fe = df[['log_gdp_origin', 'log_gdp_dest']].values

model_ppml_fe = PPML(
    endog=y_ppml_fe,
    exog=X_ppml_fe,
    entity_id=df['pair_id'].values,
    time_id=df['year'].values,
    fixed_effects=True,
    exog_names=['log_gdp_origin', 'log_gdp_dest']
)

result_ppml_fe = model_ppml_fe.fit()

print("\n" + "="*60)
print("PPML WITH PAIR FIXED EFFECTS")
print("="*60)
print(result_ppml_fe.summary())
print("\nNota: Distância foi absorvida pelos pair fixed effects")

## 6. Quando usar PPML?

### Use PPML quando:

1. **Zeros presentes** nos dados (comum em comércio, FDI, migração)
2. **Heteroskedasticidade** suspeita ou conhecida
3. **Modelos gravitacionais** de comércio, FDI, migração
4. **Dados de contagem** ou contínuos não-negativos

### Vantagens do PPML:

- ✅ **Consistente** sob heteroskedasticidade (propriedade QML)
- ✅ **Handles zeros** naturalmente (sem perda de observações)
- ✅ **Especificação correta** de $E[y|X]$
- ✅ **Interpretação** direta de elasticidades
- ✅ **Robusto** a outliers (comparado a OLS)

### Literatura:

PPML tornou-se o **método padrão** para modelos gravitacionais:
- Santos Silva & Tenreyro (2006): artigo seminal
- Head & Mayer (2014): survey de modelos gravitacionais
- Yotov et al. (2016): "An Advanced Guide to Trade Policy Analysis" (WTO/UNCTAD)

### Implementação em R:

- `gravity::ppml()` - pacote gravity
- `alpaca::feglm()` - para high-dimensional FE

### Implementação em Python:

- **PanelBox** - `PPML` (este tutorial)
- statsmodels - `Poisson` (básico, sem wrapper PPML)

## 7. Conclusões

Neste tutorial, demonstramos:

1. **PPML vs OLS(log)**:
   - PPML usa TODAS as observações (incluindo zeros)
   - PPML é consistente sob heteroskedasticidade
   - OLS(log) perde observações e é inconsistente

2. **Elasticidades**:
   - Interpretação direta dos coeficientes
   - Métodos `.elasticity()` e `.elasticities()`

3. **Fixed Effects**:
   - PPML suporta pair/time FE
   - Controla fatores não observados

4. **Best Practices**:
   - Sempre usar cluster-robust SEs (obrigatório no PanelBox PPML)
   - Preferir PPML a OLS(log) para modelos gravitacionais
   - Verificar presença de zeros e heteroskedasticidade

### Próximos passos:

- Explorar dados reais de comércio (ex: BACI, Comtrade)
- Adicionar variáveis dummy (fronteira, idioma, acordos comerciais)
- Comparar com outros estimadores (NLS, GPML)
- Estimar efeitos de políticas comerciais

In [ ]:
# Resumo final: Verdadeiros vs Estimados
summary_df = pd.DataFrame({
    'Variable': ['log(GDP origin)', 'log(GDP dest)', 'log(Distance)'],
    'True': [beta_gdp_origin, beta_gdp_dest, beta_distance],
    'PPML': [result_ppml.params[1], result_ppml.params[2], result_ppml.params[3]],
    'OLS(log)': [result_ols.params[1], result_ols.params[2], result_ols.params[3]],
    'PPML Error': [
        abs(result_ppml.params[1] - beta_gdp_origin),
        abs(result_ppml.params[2] - beta_gdp_dest),
        abs(result_ppml.params[3] - beta_distance)
    ],
    'OLS Error': [
        abs(result_ols.params[1] - beta_gdp_origin),
        abs(result_ols.params[2] - beta_gdp_dest),
        abs(result_ols.params[3] - beta_distance)
    ]
})

print("\n" + "="*80)
print("FINAL SUMMARY: Parameter Recovery")
print("="*80)
print(summary_df.to_string(index=False))
print("\n" + "="*80)
print(f"Sample size: PPML = {len(df)}, OLS = {len(df_no_zeros)} (lost {pct_zeros:.1f}% due to zeros)")
print("="*80)